# LineListMaker — Chainable Line-List Filtering & Export

This notebook demonstrates the `LineListMaker` class, a fluent builder for filtering spectral line-list data and exporting to CSV, `.par`, or DataFrame.

**Key features:**
- Accepts a `MoleculeLineList` or raw `pd.DataFrame` as input
- Every filter method returns `self` so calls can be chained
- Species label auto-derived from `molecule_id`, with manual override
- Built-in filter inspection, undo (`pop_filter`), and full `reset`
- Export to CSV, `.par`, `DataFrame`, or a new `MoleculeLineList`
- Merge / append multiple species into a single line list

In [1]:
# Imports
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display

# iSLAT imports
from iSLAT.Modules.DataTypes.MoleculeLineList import MoleculeLineList
from iSLAT.Modules.DataProcessing.LineListMaker import LineListMaker
from iSLAT.Modules.FileHandling import hitran_data_folder_path

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version:  {np.__version__}")

Pandas version: 2.3.3
NumPy version:  2.3.5


## 1. Load Molecular Data

Load a HITRAN `.par` file into a `MoleculeLineList`, then wrap it in a `LineListMaker`.

In [ ]:
# Load H2O line list from a HITRAN .par file
h2o_lines = MoleculeLineList(
    molecule_id="H2O",
    filename=hitran_data_folder_path / "data_Hitran_H2O.par"
)

# Create a LineListMaker — species is auto-derived from molecule_id
maker = LineListMaker(h2o_lines)

print(maker)                     # __repr__
print(f"Total lines: {len(maker)}")  # __len__
display(maker.df)

<LineListMaker species='H2O' lines=305561 filters=0>
Total lines: 305561


,species,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,H2O,0,0_0_0|10_2_9,0_0_0|9_3_6,933.27661,3.212257e+11,0.000006,1861.25073,1845.83411,63,57
1,H2O,1,0_0_1|5_1_5,0_0_1|4_2_2,928.22180,3.229750e+11,0.000009,5865.74316,5850.24268,33,27
2,H2O,2,0_2_0|6_5_1,0_2_0|7_4_4,926.64453,3.235247e+11,0.000026,6039.06494,6023.53857,13,15
3,H2O,3,0_1_0|14_3_12,0_1_0|13_4_9,926.56085,3.235540e+11,0.000009,6021.03809,6005.50977,87,81
4,H2O,4,0_0_0|5_1_5,0_0_0|4_2_2,922.00464,3.251529e+11,0.000012,469.94110,454.33624,11,9
5,H2O,5,1_0_0|5_1_5,1_0_0|4_2_2,917.67560,3.266867e+11,0.000010,5722.67236,5706.99365,11,9
6,H2O,6,0_2_0|3_2_1,0_2_0|4_1_4,905.37897,3.311237e+11,0.000033,4881.40918,4865.51709,21,27
7,H2O,7,0_1_0|5_2_3,0_1_0|6_1_6,891.63464,3.362279e+11,0.000011,2955.20264,2939.06616,33,39
8,H2O,8,0_0_0|16_6_11,0_0_0|17_3_14,884.22882,3.390440e+11,0.000008,5499.35889,5483.08691,99,105
9,H2O,9,0_0_0|17_4_13,0_0_0|16_7_10,844.94141,3.548086e+11,0.000009,5780.87305,5763.84570,105,99


## 2. Chainable Filtering

Every filter returns `self`, so you can chain multiple calls in a single expression. Here we select H₂O lines in the 10–20 µm range with E_up ≤ 5000 K and Einstein-A ≥ 0.01 s⁻¹.

In [ ]:
# Chain several filters in one expression
maker = (
    LineListMaker(h2o_lines)
    .filter_wavelength(min_val=10, max_val=20)
    .filter_eup(max_val=5000)
    .filter_astein(min_val=1e-2)
    .sort("lam")
)

print(f"Filtered down to {len(maker)} lines")
display(maker.df)

Filtered down to 265 lines


,species,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,H2O,7588,0_1_0|8_3_6,0_0_0|9_6_3,10.01192,2.994356e+13,0.01572,3784.26001,2347.19604,51,57
1,H2O,7504,0_0_0|13_4_9,0_0_0|12_1_12,10.24647,2.925811e+13,0.02053,3645.56250,2241.39502,81,75
2,H2O,7492,0_0_0|13_6_8,0_0_0|12_1_11,10.27239,2.918429e+13,0.07314,3953.90161,2553.27686,27,25
3,H2O,7487,0_1_0|12_2_11,0_0_0|13_3_10,10.29171,2.912951e+13,0.29250,4872.24414,3474.24780,75,81
4,H2O,7462,0_0_0|13_7_7,0_0_0|12_2_10,10.34268,2.898597e+13,0.11480,4211.40771,2820.30103,27,25
5,H2O,7459,0_1_0|9_3_7,0_0_0|10_6_4,10.35230,2.895902e+13,0.02124,4088.18433,2698.37085,19,21
6,H2O,7458,0_1_0|9_2_8,0_0_0|10_5_5,10.35316,2.895660e+13,0.01886,3871.16382,2481.46631,19,21
7,H2O,7445,0_0_0|11_11_0,0_0_0|10_8_3,10.39599,2.883732e+13,0.02128,4627.38428,3243.41113,69,63
8,H2O,7444,0_0_0|11_11_1,0_0_0|10_8_2,10.39600,2.883729e+13,0.02128,4627.38428,3243.41187,23,21
9,H2O,7432,0_1_0|12_1_11,0_0_0|13_4_10,10.41823,2.877575e+13,0.27290,4871.77344,3490.75537,25,27


## 3. Quantum-State Filtering

Use `filter_quantum()` to select lines by their upper/lower quantum labels. Set `contains=True` for substring matching — here we keep only v1-1 band lines (both labels start with `0_1_0`).

In [ ]:
# Select only v1-1 band lines (substring match on quantum labels)
v1_maker = (
    LineListMaker(h2o_lines)
    .filter_wavelength(min_val=10, max_val=20)
    .filter_quantum(lev_up="0_1_0", contains=True)
    .filter_quantum(lev_low="0_1_0", contains=True)
    .sort("lam")
)

print(f"v1-1 lines (10-20 µm): {len(v1_maker)}")
display(v1_maker.df)

v1-1 lines (10–20 µm): 471


,species,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,H2O,7587,0_1_0|11_10_1,0_1_0|10_7_4,10.01481,2.993492e+13,0.068860,6861.86133,5425.21143,69,63
1,H2O,7585,0_1_0|11_10_2,0_1_0|10_7_3,10.01494,2.993452e+13,0.068860,6861.86133,5425.23047,23,21
2,H2O,7584,0_1_0|20_4_16,0_1_0|19_3_17,10.01604,2.993123e+13,12.580000,10027.41309,8590.94141,41,39
3,H2O,7577,0_1_0|13_9_4,0_1_0|12_6_7,10.03925,2.986203e+13,0.478900,7365.63818,5932.48682,81,75
4,H2O,7567,0_1_0|13_9_5,0_1_0|12_6_6,10.06264,2.979263e+13,0.479600,7365.63721,5935.81641,27,25
5,H2O,7560,0_1_0|13_8_6,0_1_0|13_3_11,10.07834,2.974622e+13,0.001678,7009.64453,5582.05176,27,27
6,H2O,7553,0_1_0|16_7_9,0_1_0|15_4_12,10.09625,2.969345e+13,2.076000,8239.45508,6814.39453,33,31
7,H2O,7548,0_1_0|13_7_7,0_1_0|13_2_12,10.10953,2.965443e+13,0.001822,6680.78760,5257.59961,27,27
8,H2O,7542,0_1_0|8_7_1,0_1_0|7_2_6,10.12396,2.961217e+13,0.000310,4757.02100,3335.86133,17,15
9,H2O,7541,0_1_0|12_6_7,0_1_0|11_1_10,10.12637,2.960512e+13,0.081330,5932.48779,4511.66553,75,69


## 4. Inspecting State — `summary()`, `.filters`, `len()`

The maker tracks every filter you apply. Use `.summary()` for a human-readable overview, or `.filters` for the raw list of `(name, kwargs)` tuples.

In [5]:
# Pretty-print the current state
print(v1_maker.summary())
print()

# Raw filter log
for i, (name, kwargs) in enumerate(v1_maker.filters):
    print(f"  [{i}] {name}: {kwargs}")

<LineListMaker species='H2O' lines=471 filters=3>
  λ range : 10.01481 – 19.91264 µm  (471 lines)
  Active filters:
    • filter_wavelength(min_val=10, max_val=20)
    • filter_quantum(lev_up='0_1_0', lev_low=None, contains=True)
    • filter_quantum(lev_up=None, lev_low='0_1_0', contains=True)

  [0] filter_wavelength: {'min_val': 10, 'max_val': 20}
  [1] filter_quantum: {'lev_up': '0_1_0', 'lev_low': None, 'contains': True}
  [2] filter_quantum: {'lev_up': None, 'lev_low': '0_1_0', 'contains': True}


## 5. Undoing Filters — `pop_filter()` and `reset()`

Made a mistake? `pop_filter()` removes the last filter and replays the rest. `reset()` drops all filters entirely.

In [6]:
# Work on a copy so we don't disturb v1_maker
demo = v1_maker.copy()
print(f"Before pop:  {len(demo)} lines, {len(demo.filters)} filters")

# Remove the last filter (the second quantum filter on lev_low)
demo.pop_filter()
print(f"After pop:   {len(demo)} lines, {len(demo.filters)} filters")

# Full reset — back to all original lines
demo.reset()
print(f"After reset: {len(demo)} lines, {len(demo.filters)} filters")

Before pop:  471 lines, 3 filters
After pop:   799 lines, 2 filters
After reset: 305561 lines, 0 filters


## 6. Custom Filters

Use `filter_custom()` to apply any arbitrary boolean mask via a lambda or function. The `label` argument is recorded in the filter log for traceability.

In [ ]:
# Keep only lines whose Einstein-A is above the median of the current set
strong_lines = (
    LineListMaker(h2o_lines)
    .filter_wavelength(min_val=10, max_val=20)
    .filter_custom(
        lambda df: df["a_stein"] > df["a_stein"].median(),
        label="above_median_astein"
    )
    .sort("a_stein", ascending=False)
)

print(strong_lines.summary())
display(strong_lines.df)

<LineListMaker species='H2O' lines=1319 filters=2>
  λ range : 10.01115 – 19.99860 µm  (1319 lines)
  Active filters:
    • filter_wavelength(min_val=10, max_val=20)
    • filter_custom(label='above_median_astein')


,species,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,H2O,6306,0_0_0|17_17_1,0_0_0|16_16_0,14.21061,2.109638e+13,205.7,10062.63574,9050.16992,35,33
1,H2O,6305,0_0_0|17_17_0,0_0_0|16_16_1,14.21061,2.109638e+13,205.6,10062.63574,9050.16992,105,99
2,H2O,6443,0_0_0|18_16_2,0_0_0|17_15_3,13.76219,2.178378e+13,190.2,10297.83105,9252.37305,37,35
3,H2O,6444,0_0_0|18_16_3,0_0_0|17_15_2,13.76219,2.178378e+13,190.2,10297.83105,9252.37305,111,105
4,H2O,6291,0_0_0|17_16_1,0_0_0|16_15_2,14.24957,2.103871e+13,184.8,9660.12109,8650.42188,105,99
5,H2O,6290,0_0_0|17_16_2,0_0_0|16_15_1,14.24957,2.103871e+13,184.7,9660.12109,8650.42188,35,33
6,H2O,6136,0_0_0|16_16_0,0_0_0|15_15_1,14.79079,2.026886e+13,179.5,9050.16992,8077.41846,33,31
7,H2O,6137,0_0_0|16_16_1,0_0_0|15_15_0,14.79079,2.026886e+13,179.5,9050.16992,8077.41846,99,93
8,H2O,6551,0_0_0|19_15_5,0_0_0|18_14_4,13.41866,2.234146e+13,174.6,10540.89258,9468.67188,39,37
9,H2O,6552,0_0_0|19_15_4,0_0_0|18_14_5,13.41866,2.234146e+13,174.6,10540.89258,9468.67188,117,111


## 7. Overriding the Species Label

The species is auto-derived from `molecule_id`, but you can override it at construction time or change it later via the `.species` property.

In [ ]:
# Override at construction
custom = LineListMaker(h2o_lines, species="H2O_custom")
print(f"Species at init: {custom.species}")

# Change later via property
custom.species = "H2O_v1-1"
print(f"Species after:   {custom.species}")
display(custom.df[["species", "lam"]])

Species at init: H2O_custom
Species after:   H2O_v1-1


,species,lam
0,H2O_v1-1,933.27661
1,H2O_v1-1,928.22180
2,H2O_v1-1,926.64453
3,H2O_v1-1,926.56085
4,H2O_v1-1,922.00464


## 8. Export to CSV

Write the filtered line list to a CSV in the standard iSLAT format (`species, lev_up, lev_low, lam, a_stein, e_up, e_low, g_up, g_low`). Pass `extended=True` to include `xmin` / `xmax` columns.

In [ ]:
# Output to a folder next to this notebook
output_dir = Path("output").resolve()
output_dir.mkdir(exist_ok=True)

csv_path = v1_maker.to_csv(output_dir / "H2O_v1-1_10-20um.csv")
print(f"Saved {len(v1_maker)} lines → {csv_path.name}")

# Quick verify
reloaded_csv = pd.read_csv(csv_path)
print(f"Columns: {list(reloaded_csv.columns)}")
display(reloaded_csv)

Saved 471 lines → H2O_v1-1_10-20um.csv
Columns: ['species', 'lev_up', 'lev_low', 'lam', 'a_stein', 'e_up', 'e_low', 'g_up', 'g_low', 'nr', 'freq']


,species,lev_up,lev_low,lam,a_stein,e_up,e_low,g_up,g_low,nr,freq
0,H2O,0_1_0|11_10_1,0_1_0|10_7_4,10.01481,0.06886,6861.86133,5425.21143,69,63,7587,2.993492e+13
1,H2O,0_1_0|11_10_2,0_1_0|10_7_3,10.01494,0.06886,6861.86133,5425.23047,23,21,7585,2.993452e+13
2,H2O,0_1_0|20_4_16,0_1_0|19_3_17,10.01604,12.58000,10027.41309,8590.94141,41,39,7584,2.993123e+13
3,H2O,0_1_0|13_9_4,0_1_0|12_6_7,10.03925,0.47890,7365.63818,5932.48682,81,75,7577,2.986203e+13
4,H2O,0_1_0|13_9_5,0_1_0|12_6_6,10.06264,0.47960,7365.63721,5935.81641,27,25,7567,2.979263e+13


## 9. Export to `.par` File

The `.par` export delegates to `MoleculeLineList.write_par_file()`, preserving the partition function and HITRAN header. This requires the maker to have been initialised from a `MoleculeLineList` (not a raw DataFrame).

In [ ]:
# Export to .par (HITRAN format) with an optional header override
header = pd.DataFrame({
    "source": ["Filtered from data_Hitran_H2O.par — v1-1 lines, 10-20 µm"],
})

par_path = v1_maker.to_par(output_dir / "H2O_v1-1_10-20um.par", header=header)
print(f"Saved {len(v1_maker)} lines → {par_path.name}")

# Verify by reloading
reloaded_par = MoleculeLineList(molecule_id="H2O_v1-1", filename=par_path)
print(f"Reloaded lines: {len(reloaded_par)}")
display(reloaded_par.get_pandas_table())

Saved 471 lines → H2O_v1-1_10-20um.par
[CACHE MISS] Parsing H2O_v1-1 from source file...
Molar_mass: 18.010565
[CACHE SAVED] H2O_v1-1 cached for faster loading
Reloaded lines: 471


,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,7587,0_1_0|11_10_1,0_1_0|10_7_4,10.01481,2.993492e+13,0.06886,6861.86133,5425.21143,69,63
1,7585,0_1_0|11_10_2,0_1_0|10_7_3,10.01494,2.993452e+13,0.06886,6861.86133,5425.23047,23,21
2,7584,0_1_0|20_4_16,0_1_0|19_3_17,10.01604,2.993123e+13,12.58000,10027.41309,8590.94141,41,39
3,7577,0_1_0|13_9_4,0_1_0|12_6_7,10.03925,2.986203e+13,0.47890,7365.63818,5932.48682,81,75
4,7567,0_1_0|13_9_5,0_1_0|12_6_6,10.06264,2.979263e+13,0.47960,7365.63721,5935.81641,27,25


## 11. Convert Back to a `MoleculeLineList`

Use `to_linelist()` to create a new `MoleculeLineList` from the filtered data. This is useful when you want to feed the result back into other iSLAT processing steps.

In [11]:
# Convert filtered data back to a MoleculeLineList object
new_linelist = v1_maker.to_linelist()

print(f"Type:        {type(new_linelist).__name__}")
print(f"molecule_id: {new_linelist.molecule_id}")
print(f"Lines:       {len(new_linelist)}")

Type:        MoleculeLineList
molecule_id: H2O
Lines:       471


## 12. Merging Multiple Species

Load a second molecule and use `LineListMaker.merge()` to combine filtered line lists from different species into a single multi-species line list.

In [ ]:
# Load a second species
co_lines = MoleculeLineList(
    molecule_id="CO",
    filename=hitran_data_folder_path / "data_HITEMP_2019_CO.par"
)

# Filter each species independently
h2o_maker = (
    LineListMaker(h2o_lines)
    .filter_wavelength(min_val=4.5, max_val=5.5)
    .filter_eup(max_val=6000)
)

co_maker = (
    LineListMaker(co_lines)
    .filter_wavelength(min_val=4.5, max_val=5.5)
    .filter_eup(max_val=6000)
)

print(f"H2O lines: {len(h2o_maker)}")
print(f"CO  lines: {len(co_maker)}")

# Merge into a single multi-species maker
combined = LineListMaker.merge(h2o_maker, co_maker)
print(f"\nCombined:  {len(combined)} lines")
display(combined.df)

H2O lines: 925
CO  lines: 56

Combined:  981 lines


,species,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,H2O,14851,0_1_0|8_6_3,0_0_0|9_3_6,5.49993,5.450840e+13,5.124000e-04,4461.82471,1845.83411,51,57
1,H2O,14854,1_0_0|2_0_2,0_1_0|3_3_1,5.49907,5.451698e+13,3.587000e-03,5360.79883,2744.39673,5,7
2,H2O,14869,0_1_0|6_0_6,0_0_0|4_0_4_q,5.49533,5.455407e+13,3.044000e-07,2937.66626,319.48441,13,9
3,H2O,14883,1_0_0|4_2_3,0_1_0|5_3_2,5.49203,5.458683e+13,2.111000e-01,5685.06006,3065.30591,27,33
4,H2O,14890,0_1_0|10_3_7,0_0_0|9_4_6,5.49036,5.460342e+13,2.583000e+00,4549.78418,1929.23413,21,19
5,H2O,14900,0_2_0|7_3_5,0_1_0|6_2_4,5.48720,5.463489e+13,7.004000e+00,5803.47119,3181.40991,15,13
6,H2O,14906,0_1_0|10_1_9,0_0_0|9_2_8,5.48618,5.464499e+13,9.508000e+00,4176.97900,1554.43347,21,19
7,H2O,14911,1_0_0|4_1_4,0_1_0|5_2_3,5.48364,5.467035e+13,4.779000e-02,5578.96484,2955.20264,27,33
8,H2O,14913,0_2_0|4_4_0,0_1_0|5_1_5,5.48262,5.468055e+13,2.475000e-04,5390.77393,2766.52173,9,11
9,H2O,14914,1_0_0|5_2_3,0_1_0|6_3_4,5.48183,5.468837e+13,2.135000e-01,5893.11523,3268.48730,33,39


## 13. Export the Merged List

Save the combined multi-species line list as a CSV. You can also use `filter_species()` on a merged maker to re-select a subset later.

In [ ]:
# Save the merged list
merged_csv = combined.sort("lam").to_csv(output_dir / "H2O_CO_4p5-5p5um.csv")
print(f"Saved {len(combined)} lines → {merged_csv.name}")

# Filter the merged maker to just CO
co_only = combined.copy().filter_species("CO").sort("lam")
print(f"\nCO-only subset: {len(co_only)} lines")
display(co_only.df)

Saved 981 lines → H2O_CO_4p5-5p5um.csv

CO-only subset: 56 lines


,species,nr,lev_up,lev_low,lam,freq,a_stein,e_up,e_low,g_up,g_low
0,CO,6489,1|R_22,0|R_22,4.50096,6.660634e+13,19.59,4593.96172,1397.37989,47,45
1,CO,6479,1|R_21,0|R_21,4.50705,6.651632e+13,19.48,4468.30888,1276.04738,45,43
2,CO,6469,1|R_20,0|R_20,4.51324,6.642515e+13,19.37,4348.08446,1160.19830,43,41
3,CO,6459,1|R_19,0|R_19,4.51952,6.633284e+13,19.26,4233.29301,1049.83698,41,39
4,CO,6439,1|R_18,0|R_18,4.52589,6.623939e+13,19.15,4123.93904,944.96772,39,37


## 14. Extended CSV & Extra Columns

Pass `extended=True` to include `xmin` / `xmax` columns (filled with `NaN` by default). You can also inject arbitrary extra columns via `extra_columns`.

In [ ]:
# Extended format with extra "note" column
ext_path = v1_maker.to_csv(
    output_dir / "H2O_v1-1_extended.csv",
    extended=True,
    extra_columns={"note": ""},
)

reloaded_ext = pd.read_csv(ext_path)
print(f"Columns: {list(reloaded_ext.columns)}")
display(reloaded_ext)

Columns: ['species', 'lev_up', 'lev_low', 'lam', 'a_stein', 'e_up', 'e_low', 'g_up', 'g_low', 'xmin', 'xmax', 'note', 'nr', 'freq']


,species,lev_up,lev_low,lam,a_stein,e_up,e_low,g_up,g_low,xmin,xmax,note,nr,freq
0,H2O,0_1_0|11_10_1,0_1_0|10_7_4,10.01481,0.06886,6861.86133,5425.21143,69,63,NaN,NaN,NaN,7587,2.993492e+13
1,H2O,0_1_0|11_10_2,0_1_0|10_7_3,10.01494,0.06886,6861.86133,5425.23047,23,21,NaN,NaN,NaN,7585,2.993452e+13
2,H2O,0_1_0|20_4_16,0_1_0|19_3_17,10.01604,12.58000,10027.41309,8590.94141,41,39,NaN,NaN,NaN,7584,2.993123e+13
3,H2O,0_1_0|13_9_4,0_1_0|12_6_7,10.03925,0.47890,7365.63818,5932.48682,81,75,NaN,NaN,NaN,7577,2.986203e+13
4,H2O,0_1_0|13_9_5,0_1_0|12_6_6,10.06264,0.47960,7365.63721,5935.81641,27,25,NaN,NaN,NaN,7567,2.979263e+13


## 15. Cleanup

Remove the example output files created by this notebook.

In [15]:
'''import shutil

# Remove the output directory and all files we created
if output_dir.exists():
    shutil.rmtree(output_dir)
    print("Cleaned up output/ directory.")
else:
    print("Nothing to clean up.")'''

'import shutil\n\n# Remove the output directory and all files we created\nif output_dir.exists():\n    shutil.rmtree(output_dir)\n    print("Cleaned up output/ directory.")\nelse:\n    print("Nothing to clean up.")'